In [6]:
import pandas as pd
import os

# 1. 데이터 읽기
file_path = 'tomato.csv'  # 파일 경로
data = pd.read_csv(file_path)

# 2. 날짜와 시간 분리
# 날짜 형식 지정 (일/월/연도)
date_format = "%d/%m/%Y %H:%M:%S"
data['수집일자'] = pd.to_datetime(data['수집일자'], format=date_format, errors='coerce')  # 날짜 변환
data['날짜'] = data['수집일자'].dt.date             # 날짜 분리
data['시간'] = data['수집일자'].dt.time             # 시간 분리

# 결측치가 발생한 경우 확인
missing_dates = data['수집일자'].isna().sum()
if missing_dates > 0:
    print(f"날짜 변환 실패: {missing_dates}개의 레코드에서 날짜 변환에 실패했습니다.")
    data = data.dropna(subset=['수집일자'])  # 변환 실패한 레코드 제거

# 3. 데이터 정렬 (수집일자 기준)
data = data.sort_values(by='수집일자').reset_index(drop=True)

# 4. 이상치 탐지 함수
def detect_outliers(df, column, threshold=3):
    """
    IQR 방식으로 이상치를 탐지하는 함수.
    :param df: 데이터프레임
    :param column: 확인할 열 이름
    :param threshold: 이상치를 정의할 표준값 (기본값 3)
    :return: 이상치 인덱스 리스트
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)].index
    return outliers

# 5. 이상치 탐지 및 제거
columns_to_check = ['건구 온도', '습구 온도', '근권', '상대 습도', '절대 습도', '이슬점', '수분 부족', '이산화탄소']
for column in columns_to_check:
    outlier_indices = detect_outliers(data, column)
    print(f"{column} 이상치 수: {len(outlier_indices)}")
    data = data.drop(outlier_indices)  # 이상치 제거

# 6. 정리된 데이터 저장
processed_file_path = 'processed_tomato_data.csv'  # 현재 디렉토리에 저장
# 또는 절대 경로를 사용할 경우:
# processed_file_path = '/path/to/your/directory/processed_tomato_data.csv'

# 경로가 없는 경우 디렉토리 생성
output_dir = os.path.dirname(processed_file_path)
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 데이터 저장
data.to_csv(processed_file_path, index=False)
print(f"전처리 완료: 처리된 데이터를 '{processed_file_path}'로 저장했습니다.")


건구 온도 이상치 수: 0
습구 온도 이상치 수: 0
근권 이상치 수: 0
상대 습도 이상치 수: 548
절대 습도 이상치 수: 0
이슬점 이상치 수: 0
수분 부족 이상치 수: 1313
이산화탄소 이상치 수: 0
전처리 완료: 처리된 데이터를 'processed_tomato_data.csv'로 저장했습니다.


In [8]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# 1. 데이터 읽기
file_path = 'tomato.csv'  # 파일 경로
data = pd.read_csv(file_path)

# 2. 날짜와 시간 분리
# 날짜 형식 지정 (일/월/연도)
date_format = "%d/%m/%Y %H:%M:%S"
data['수집일자'] = pd.to_datetime(data['수집일자'], format=date_format, errors='coerce')  # 날짜 변환
data['날짜'] = data['수집일자'].dt.date             # 날짜 분리
data['시간'] = data['수집일자'].dt.time             # 시간 분리

# 결측치가 발생한 경우 확인
missing_dates = data['수집일자'].isna().sum()
if missing_dates > 0:
    print(f"날짜 변환 실패: {missing_dates}개의 레코드에서 날짜 변환에 실패했습니다.")
    data = data.dropna(subset=['수집일자'])  # 변환 실패한 레코드 제거

# 3. 데이터 정렬 (수집일자 기준)
data = data.sort_values(by='수집일자').reset_index(drop=True)

# 4. 이상치 탐지 함수
def detect_outliers(df, column, threshold=3):
    """
    IQR 방식으로 이상치를 탐지하는 함수.
    :param df: 데이터프레임
    :param column: 확인할 열 이름
    :param threshold: 이상치를 정의할 표준값 (기본값 3)
    :return: 이상치 인덱스 리스트
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)].index
    return outliers

# 5. 이상치 탐지 및 제거
columns_to_check = ['건구 온도', '습구 온도', '근권', '상대 습도', '절대 습도', '이슬점', '수분 부족', '이산화탄소']
for column in columns_to_check:
    outlier_indices = detect_outliers(data, column)
    print(f"{column} 이상치 수: {len(outlier_indices)}")
    data = data.drop(outlier_indices)  # 이상치 제거

# 6. 특성 선택 및 스케일링
# 예시로 '건구 온도', '습구 온도', '상대습도', '이산화탄소'가 특성이라고 가정
features = data[['건구 온도', '습구 온도', '상대 습도','절대 습도', '이산화탄소']]  # 예시로 특성 선택
target = data['품질']  # 품질이 타겟 변수로 설정

# 7. 특성 스케일링
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# 8. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

# 9. 모델 훈련
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# 10. 예측 및 평가
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# 11. 하이퍼파라미터 튜닝 (GridSearch)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# 12. 최적 모델 출력
best_model = grid_search.best_estimator_
print(f"최적 하이퍼파라미터: {grid_search.best_params_}")

# 13. 최적 모델로 예측 및 평가
y_pred_best = best_model.predict(X_test)
mse_best = mean_squared_error(y_test, y_pred_best)
print(f"최적 모델의 Mean Squared Error: {mse_best}")

# 14. 중요 특성 확인
feature_importances = best_model.feature_importances_
for feature, importance in zip(features.columns, feature_importances):
    print(f'{feature}: {importance}')

# 15. 결과 시각화
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred_best)
plt.xlabel('실제 품질')
plt.ylabel('예측 품질')
plt.title('실제 품질 vs 예측 품질')
plt.show()

# 16. 최종 모델 저장 (선택 사항)
import joblib
joblib.dump(best_model, 'tomato_quality_predictor_model.pkl')
print("모델이 저장되었습니다.")


건구 온도 이상치 수: 0
습구 온도 이상치 수: 0
근권 이상치 수: 0
상대 습도 이상치 수: 548
절대 습도 이상치 수: 0
이슬점 이상치 수: 0
수분 부족 이상치 수: 1313
이산화탄소 이상치 수: 0


KeyError: '품질'